In [1]:
!pip install torch accelerate bitsandbytes datasets transformers peft trl scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.3 MB/s eta 0:00:00


In [2]:
!nvidia-smi

Sun Jul 30 13:59:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import argparse
import bitsandbytes as bnb
from datasets import load_dataset
from functools import partial
import os
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, Trainer, TrainingArguments, BitsAndBytesConfig, \
    DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset
from torch import cuda, bfloat16
import transformers

import torch
import torch.nn as nn

***Get Model***

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
#model_id = 'meta-llama/Llama-2-13b-chat-hf'
model_id = "meta-llama/Llama-2-7b-hf"

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'


In [6]:
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# begin initializing HF items, need auth token for these

model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=True
)

In [7]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=True
)
model.eval()
print(f"Model loaded on {device}")

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded on cuda:0


In [8]:
mem = model.get_memory_footprint()
print("Memory footprint: {} ".format(mem))

# should be (7B) 7,000,000,000*4(Int4) / 8(8 bits per byte) = 3,500,000,000 = 3.5GB
# actual (7B)  3,829,940,224 (not all weights become int 4)
# actual (13B)  7,083,970,560  (not all weights become int 4)

Memory footprint: 3829936128 


In [10]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=True
)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


***Get Dataset***

In [11]:
# Load the dataset from Hugging Face
dataset = load_dataset("pubmed_qa", "pqa_labeled", split="train")

# ref: https://huggingface.co/datasets/pubmed_qa
# you can use any of the following config names as a second argument: "pqa_artificial", "pqa_labeled", "pqa_unlabeled"
# pqa_labeled has been hand labeled
# Link to Paper: https://arxiv.org/abs/1909.06146

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [12]:
print(f'Number of records: {len(dataset)}')
print(f'Column names are: {dataset.column_names}')

# ['pubid', 'question', 'context', 'long_answer', 'final_decision']

Number of records: 1000
Column names are: ['pubid', 'question', 'context', 'long_answer', 'final_decision']


In [14]:
#only the ones where the answers have been labeled as being present
dataset = dataset.filter(lambda example: example['final_decision'] == "yes")
print(f'Number of records: {len(dataset)}')
print(f'Column names are: {dataset.column_names}')

Filter:   0%|          | 0/552 [00:00<?, ? examples/s]

Number of records: 552
Column names are: ['pubid', 'question', 'context', 'long_answer', 'final_decision']


In [17]:
def create_prompt(rec):
    contactenated_context = "".join(rec['context']["contexts"])

    start = "Answer the below question based on the context provided."
    question = f"### QUESTION:\n{rec['question']}\n\n"
    context = f"Context:\n {contactenated_context}"
    response = f"### ANSWER:\n{rec['long_answer']}\n\n"
    end = "### End"

    parts = [part for part in [start, question, context, response, end] if part]

    formatted_prompt = "\n\n".join(parts)
    formatted_prompt = formatted_prompt.replace('\\n', '\n')

    rec["text"] = formatted_prompt

    return rec

In [18]:
p = create_prompt(dataset[30])
print(p)
print(p["text"])

{'pubid': 24237112, 'question': "Do provider service networks result in lower expenditures compared with HMOs or primary care case management in Florida's Medicaid program?", 'context': {'contexts': ["To determine the impact of Florida's Medicaid Demonstration 4 years post-implementation on per member per month (PMPM) Medicaid expenditures and whether receiving care through HMOs versus provider service networks (PSNs) in the Demonstration was associated with PMPM expenditures.DATA: Florida Medicaid claims from two fiscal years prior to implementation of the Demonstration (FY0405, FY0506) and the first four fiscal years after implementation (FY0607-FY0910) from two urban Demonstration counties and two urban non-Demonstration counties.", 'A difference-in-difference approach was used to compare changes in enrollee expenditures before and after implementation of the Demonstration overall and specifically for HMOs and PSNs.', 'Claims data were extracted for enrollees in the Demonstration an

In [19]:
dataset = dataset.map(create_prompt)

Map:   0%|          | 0/552 [00:00<?, ? examples/s]

In [20]:
dataset = dataset.map(
        batched=True,
        remove_columns=['pubid', 'question', 'context', 'long_answer', 'final_decision']
    )

Map:   0%|          | 0/552 [00:00<?, ? examples/s]

In [36]:
print(dataset[30]["text"])

Answer the below question based on the context provided.

### QUESTION:
The Omega-3 Index: a new risk factor for death from coronary heart disease?



Context:
 Low intakes or blood levels of eicosapentaenoic and docosahexaenoic acids (EPA + DHA) are independently associated with increased risk of death from coronary heart disease (CHD). In randomized secondary prevention trials, fish or fish oil have been demonstrated to reduce total and CHD mortality at intakes of about 1 g/day. Red blood cell (RBC) fatty acid (FA) composition reflects long-term intake of EPA + DHA. We propose that the RBC EPA + DHA (hereafter called the Omega-3 Index) be considered a new risk factor for death from CHD.We conducted clinical and laboratory experiments to generate data necessary for the validation of the Omega-3 Index as a CHD risk predictor. The relationship between this putative marker and risk for CHD death, especially sudden cardiac death (SCD), was then evaluated in several published primary and s

In [ ]:
#Save dataset to the hub for future use
#dataset.push_to_hub("Venkat-Ram-Rao/processed_pubmed_qa_dataset", private=True)

In [23]:
#max length of the model
def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length

In [24]:
mx = get_max_length(model)
mx

Found max lenth: 4096


4096

In [25]:
len(dataset)

552

In [26]:
#tokenize dataset
dataset = dataset.map(lambda samples: tokenizer(samples['text']), batched=True)

Map:   0%|          | 0/552 [00:00<?, ? examples/s]

In [27]:
len(dataset)

552

In [28]:
dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < mx)

Filter:   0%|          | 0/552 [00:00<?, ? examples/s]

In [29]:
len(dataset)

552

In [30]:
seed = 42
set_seed(seed)

dataset = dataset.shuffle(seed=seed)

***Freeze Original Weights***

In [31]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

***Create Lora Config***

In [32]:
def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names:  # needed for 16-bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

In [33]:
modules = find_all_linear_names(model)
print(modules)

#['v_proj', 'up_proj', 'down_proj', 'k_proj', 'o_proj', 'q_proj', 'gate_proj']

['o_proj', 'q_proj', 'k_proj', 'up_proj', 'v_proj', 'gate_proj', 'down_proj']


In [34]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,  #attention heads
    lora_alpha=64,  #alpha scaling
    target_modules=modules,  #gonna train all
    lora_dropout=0.1,  # dropout probability for layers
    bias="none",
    task_type="CAUSAL_LM", #for Decoder models like GPT Seq2Seq for Encoder-Decoder models like T5
)



In [35]:
##Get the PEFT Model using the downloaded model and the loRA config
model = get_peft_model(model, config)

***Training***

In [37]:
# Print Trainable parameters
trainable_params = 0
all_param = 0
for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
        trainable_params += param.numel()
print(
    f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
)

trainable params: 39976960 || all params: 3540389888 || trainable%: 1.1291682911958425


In [38]:
tokenizer.pad_token = tokenizer.eos_token

In [39]:
trainer = Trainer(
    model=model,
    train_dataset=dataset,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        max_steps=50, #20,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
    ),
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

model.config.use_cache = False  # re-enable for inference to speed up predictions for similar inputs

trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,1.664600
2,1.451400
3,1.456100
4,1.415400
5,1.749100
6,1.354800
7,1.571700
8,1.431900
9,1.412900
10,1.320700


TrainOutput(global_step=50, training_loss=1.3867300391197204, metrics={'train_runtime': 618.0213, 'train_samples_per_second': 0.324, 'train_steps_per_second': 0.081, 'total_flos': 2055859598278656.0, 'train_loss': 1.3867300391197204, 'epoch': 0.36})

***Push to Hub***

In [40]:
model.push_to_hub("Venkat-Ram-Rao/Llama2_7B_qlora_pubmed",
                  use_auth_token=True,
                  commit_message="fine tuned on kaist-ai/CoT-Collection",
                  private=True)

adapter_model.bin:   0%|          | 0.00/160M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Venkat-Ram-Rao/Llama2_7B_qlora_pubmed/commit/6b0ad53d9e02a66938e4c113aff9ad43d55d33dc', commit_message='fine tuned on kaist-ai/CoT-Collection', commit_description='', oid='6b0ad53d9e02a66938e4c113aff9ad43d55d33dc', pr_url=None, pr_revision=None, pr_num=None)

***Load from Hub***

In [41]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "Venkat-Ram-Rao/Llama2_7B_qlora_pubmed"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [42]:
mem = model.get_memory_footprint()
print("Memory footprint: {} ".format(mem))

#7,227,846,656

Memory footprint: 7227846656 


***Inference***

In [43]:
tst = """Answer the below question based on the context provided.

### QUESTION:
What can Product X be used to treat?



Context:
Product X is used to treat migraines and nausea. It is well tolerated in patients from above 15 years of age. Adverse Events include some rashes and restless leg syndrome.


### ANSWER:"""

In [44]:
batch = tokenizer(tst, return_tensors='pt')



In [46]:
with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=90)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))





 Answer the below question based on the context provided.

### QUESTION:
What can Product X be used to treat?



Context:
Product X is used to treat migraines and nausea. It is well tolerated in patients from above 15 years of age. Adverse Events include some rashes and restless leg syndrome.


### ANSWER:
Product X is a safe and effective treatment for migraines and nausea.



### Endpoint:
Product X is a safe and effective treatment for migraines and nausea.



### Endpoint:
Product X is a safe and effective treatment for migraines and nausea.



### Endpoint:
Product X is a safe and effective treatment


Test2

In [48]:
tst = """Answer the below question based on the context provided.

### QUESTION:
If I take Product X, will I get a rash?

Context:
Product X is used to treat migraines and nausea. It is well tolerated in patients from above 15 years of age. Adverse Events include some rashes and restless leg syndrome.

### ANSWER:"""

batch = tokenizer(tst, return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=90)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 Answer the below question based on the context provided.

### QUESTION:
If I take Product X, will I get a rash?

Context:
Product X is used to treat migraines and nausea. It is well tolerated in patients from above 15 years of age. Adverse Events include some rashes and restless leg syndrome.

### ANSWER:
Product X is well tolerated in patients from above 15 years of age. Adverse Events include some rashes and restless leg syndrome.



### Endpoint:
Product X is well tolerated in patients from above 15 years of age. Adverse Events include some rashes and restless leg syndrome.



### Endpoint:
Product X is well toler


In [49]:
tst = """Answer the below question based on the context provided.

### QUESTION:
I am 9, can I take Product X?

Context:
Product X is used to treat migraines and nausea. It is well tolerated in patients from above 15 years of age. Adverse Events include some rashes and restless leg syndrome.

### ANSWER:"""

batch = tokenizer(tst, return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=90)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 Answer the below question based on the context provided.

### QUESTION:
I am 9, can I take Product X?

Context:
Product X is used to treat migraines and nausea. It is well tolerated in patients from above 15 years of age. Adverse Events include some rashes and restless leg syndrome.

### ANSWER:
Product X is not recommended for children under 15 years of age.



### Endpoint:
Product X is not recommended for children under 15 years of age.



### Endpoint:
Product X is not recommended for children under 15 years of age.



### Endpoint:
Product X is not recommended for children under 15 years of age
